# Compare zero-crossings to auto-detected sniff inhalation onsets
### Get zero crossings

In [1]:
import zerocrossing
rootpath = "./data/190603/"
files = ["1953_1_04_D2019_6_3T12_29_13_odor.h5"]
# files = ["1953_1_05_D2019_6_3T12_47_22_odor.h5"]
crossings = zerocrossing.getCrossingsFromFiles(files, rootpath)
# Only one file, so let's get rid of the dict
print(crossings)
crossings = crossings[files[0]]

{'1953_1_04_D2019_6_3T12_29_13_odor.h5': array([False, False, False, ..., False, False, False])}


In [2]:
print(f"len(crossings) = {len(crossings)}")
# [[len(x) for x in crossing[:5]] for crossing in crossings]

len(crossings) = 391406


In [3]:
# [(len(x.nonzero()[0])) for x in crossings[1]]

In [4]:
# print(f"All events have exactly one crossing? {all([all([(len(x.nonzero()[0]) == 1) for x in y]) for y in crossings])}")

In [5]:
# Crossing positions
print(crossings.nonzero()[0])
print(len(crossings.nonzero()[0]))

[   121    272    427 ... 390494 391091 391393]
1140


### Get Events data to assign timestamp to zero crossings

In [8]:
import h5py
sniffIndexes = zerocrossing.getSniffIndexesByTrialEvents(rootpath + files[0])
crossingTimestamps = []
with h5py.File(rootpath + files[0], "r") as h5File:
    numFrameTriggers = sum([sum([len(x) for x in h5File["Trial{:04d}".format(i_trial)]["frame_triggers"]]) for i_trial in range(1,46)])
    print(f"Total frame triggers: {numFrameTriggers}")
    for i_trial, trialSniffIndexes in enumerate(sniffIndexes):
        events = h5File["Trial{:04d}".format(i_trial+1)]["Events"]
        timestamps = []
        for i_sample, sampleSniffIndexes in enumerate(trialSniffIndexes):
            sample = crossings[sampleSniffIndexes]
            crossingIndexes = sample.nonzero()[0]
            # Need negative offsets -- matching Events packet_sent_time is ending timestamp
            offsets = crossingIndexes - len(sample)
            timestamps.append(events["packet_sent_time"][i_sample] + offsets)
            if i_sample > 0:
                # confirm 1kHz sample-rate
                assert (len(sample) + events["packet_sent_time"][i_sample - 1]) == events["packet_sent_time"][i_sample]
        crossingTimestamps.append(timestamps)


Total frame triggers: 11679


In [15]:
print(len(crossingTimestamps[15][0]))
print([max([len(x) for x in y]) for y in crossingTimestamps])
print([len(x) for x in crossingTimestamps[15]])

0
[1, 3, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]


### Compare crossings to auto-detected inhalation onset timestamps

In [16]:
exactMatches = []
with h5py.File(rootpath + files[0], "r") as h5File:
    trials = h5File['Trials']
    inhalationOnsets = trials['inh_onset']
    for i_trial, onsetTimestamp in enumerate(inhalationOnsets):
        curCrossings = crossingTimestamps[i_trial]
        exactMatches.append(any([onsetTimestamp in eventCrossings for eventCrossings in curCrossings]))
print(f"Any matches? {any(exactMatches)}")

Any matches? True


In [17]:
import numpy as np
print(f"{sum(exactMatches)} matches out of {len(exactMatches)} trials, at trial numbers {np.nonzero(exactMatches)[0]}.")

2 matches out of 45 trials, at trial numbers [36 39].


### Get frames corresponding to each timestamp

In [ ]:
frameTimestamps = []
with h5py.File(rootpath + files[0], "r") as h5File:
    for i_trial, crossing in enumerate(crossings[1:]):
        frameTriggers = h5File["Trial{:04d}".format(i_trial+1+1)]["frame_triggers"]
        print(len(frameTriggers))
        print(type(frameTriggers))
        print(type(frameTriggers[0]))
#         print(frameTriggers[0])
